In [1]:
import pandas as pd
import numpy as np
import os
import json
import itertools as it
import time
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from datetime import date, datetime, timedelta
from tensorflow import keras
from sklearn.metrics import confusion_matrix,classification_report,ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings("ignore")

2024-01-24 08:19:08.112141: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-24 08:19:08.360462: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-24 08:19:09.183317: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0
majestic

,domain,label,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,google.com,0,103,111,111,103,108,101,46,99,...,0,0,0,0,0,0,0,0,0,0
1,facebook.com,0,102,97,99,101,98,111,111,107,...,0,0,0,0,0,0,0,0,0,0
2,youtube.com,0,121,111,117,116,117,98,101,46,...,0,0,0,0,0,0,0,0,0,0
3,twitter.com,0,116,119,105,116,116,101,114,46,...,0,0,0,0,0,0,0,0,0,0
4,instagram.com,0,105,110,115,116,97,103,114,97,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,fidgit.com,0,102,105,100,103,105,116,46,99,...,0,0,0,0,0,0,0,0,0,0
999996,fixwindows10connections.com,0,102,105,120,119,105,110,100,111,...,0,0,0,0,0,0,0,0,0,0
999997,flyerguide.com,0,102,108,121,101,114,103,117,105,...,0,0,0,0,0,0,0,0,0,0
999998,forwardpathway.com,0,102,111,114,119,97,114,100,112,...,0,0,0,0,0,0,0,0,0,0


In [3]:
dga = pd.read_csv('./treino/df_train.csv')
dga = dga.drop('family', axis = 1)
dga

,domain,label,3,4,5,6,7,8,9,10,...,63,64,65,66,67,68,69,70,71,72
0,uzwdmrlhr.tk,1,117,122,119,100,109,114,108,104,...,0,0,0,0,0,0,0,0,0,0
1,uzwdmrlhr.com,1,117,122,119,100,109,114,108,104,...,0,0,0,0,0,0,0,0,0,0
2,uzwdmrlhr.pages.dev,1,117,122,119,100,109,114,108,104,...,0,0,0,0,0,0,0,0,0,0
3,rhrlwdumz.tk,1,114,104,114,108,119,100,117,109,...,0,0,0,0,0,0,0,0,0,0
4,rhrlwdumz.com,1,114,104,114,108,119,100,117,109,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352254,kwgjjpbcaaanbivklyxx.com,1,107,119,103,106,106,112,98,99,...,0,0,0,0,0,0,0,0,0,0
1352255,vxijfvgvihjnfvofxmyc.com,1,118,120,105,106,102,118,103,118,...,0,0,0,0,0,0,0,0,0,0
1352256,cepswmbgtknhtkhlsecg.com,1,99,101,112,115,119,109,98,103,...,0,0,0,0,0,0,0,0,0,0
1352257,tifjhjtouqmdlbkfnuow.com,1,116,105,102,106,104,106,116,111,...,0,0,0,0,0,0,0,0,0,0


In [4]:
dga.columns = majestic.columns

In [5]:
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

In [6]:
df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)
df_treino

,domain,label,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,wwoof.fr,0,119,119,111,111,102,46,102,114,...,0,0,0,0,0,0,0,0,0,0
1,digimanthan.com,0,100,105,103,105,109,97,110,116,...,0,0,0,0,0,0,0,0,0,0
2,injo.com,0,105,110,106,111,46,99,111,109,...,0,0,0,0,0,0,0,0,0,0
3,aclanthology.info,0,97,99,108,97,110,116,104,111,...,0,0,0,0,0,0,0,0,0,0
4,aseabreeze.co.uk,0,97,115,101,97,98,114,101,101,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852202,kwgjjpbcaaanbivklyxx.com,1,107,119,103,106,106,112,98,99,...,0,0,0,0,0,0,0,0,0,0
1852203,vxijfvgvihjnfvofxmyc.com,1,118,120,105,106,102,118,103,118,...,0,0,0,0,0,0,0,0,0,0
1852204,cepswmbgtknhtkhlsecg.com,1,99,101,112,115,119,109,98,103,...,0,0,0,0,0,0,0,0,0,0
1852205,tifjhjtouqmdlbkfnuow.com,1,116,105,102,106,104,106,116,111,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

In [9]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

In [10]:
X_train.shape

(1852207, 70)

In [11]:
#Carregando hyperparâmetros de regularizadores, otimizador e função de ativação
l1l2 = keras.regularizers.l1_l2(l1=0.00001, l2=0.0001)
opt = keras.optimizers.Adam(learning_rate=0.0001)
fx = "relu"

2024-01-22 16:01:53.675400: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-22 16:01:53.730169: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-22 16:01:53.730319: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [12]:
#Estrutura do modelo convolucional profundo
model_bin = tf.keras.Sequential([
      
        layers.Embedding(input_dim=256,output_dim=60,input_length=70),
    
        layers.Conv1D(500, 7, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
            
        layers.Conv1D(250, 6, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
        layers.MaxPooling1D(pool_size=6, strides=1, padding='same'),
    
        layers.Conv1D(120, 5, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
    
        layers.Conv1D(60, 5, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
        layers.MaxPooling1D(pool_size=4, strides=1, padding='same'),
    
        layers.Conv1D(30, 4, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
    
        layers.Conv1D(30, 3, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
        layers.MaxPooling1D(pool_size=3, strides=1, padding='same'),
    
        layers.Flatten(),
        layers.Dense(15, activation=fx, activity_regularizer=l1l2),
        layers.Dense(1, activation='sigmoid')])

model_bin.compile(optimizer = opt, loss = 'binary_crossentropy', metrics='accuracy')
model_bin.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 70, 60)            15360     
                                                                 
 conv1d (Conv1D)             (None, 70, 500)           210500    
                                                                 
 re_lu (ReLU)                (None, 70, 500)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 70, 250)           750250    
                                                                 
 re_lu_1 (ReLU)              (None, 70, 250)           0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 70, 250)           0         
 D)                                                              
                                                        

In [13]:
best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

In [7]:
# Ajustando uso de memoria RAM e GPU
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  pass

2024-01-23 13:53:31.576393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-23 13:53:31.762714: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-23 13:53:31.763041: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [15]:
model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10


2024-01-22 16:02:07.688802: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2024-01-22 16:02:08.330287: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-01-22 16:02:08.340279: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f2f24004a10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-22 16:02:08.340297: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-01-22 16:02:08.344146: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-22 16:02:08.423708: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

7408/7409 [============================>.] - ETA: 0s - loss: 0.1030 - accuracy: 0.9635

2024-01-22 16:06:51.466106: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.24GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-22 16:06:51.616001: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.86GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-22 16:06:51.736161: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.70GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-22 16:06:51.882202: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU

7409/7409 [==============================] - ETA: 0s - loss: 0.1030 - accuracy: 0.9635

...

7409/7409 [==============================] - 303s 41ms/step - loss: 0.0200 - accuracy: 0.9937 - val_loss: 0.0796 - val_accuracy: 0.9741
Epoch 9/10
7409/7409 [==============================] - ETA: 0s - loss: 0.0172 - accuracy: 0.9947INFO:tensorflow:Assets written to: best_model_acc/assets


INFO:tensorflow:Assets written to: best_model_acc/assets


7409/7409 [==============================] - 303s 41ms/step - loss: 0.0172 - accuracy: 0.9947 - val_loss: 0.1132 - val_accuracy: 0.9656
Epoch 10/10
7409/7409 [==============================] - ETA: 0s - loss: 0.0149 - accuracy: 0.9955INFO:tensorflow:Assets written to: best_model_acc/assets


INFO:tensorflow:Assets written to: best_model_acc/assets


7409/7409 [==============================] - 302s 41ms/step - loss: 0.0149 - accuracy: 0.9955 - val_loss: 0.1277 - val_accuracy: 0.9650


In [16]:
model_bin.load_weights('best_model_acc')

2024-01-22 16:52:40.854717: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open best_model_acc: FAILED_PRECONDITION: best_model_acc; É um diretório: perhaps your file is in a different file format and you need to use a different restore operator?


In [17]:
model_bin = tf.keras.models.load_model('best_model_acc')

In [18]:
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

## Testes Treinamento Inicial

In [19]:
model_bin = tf.keras.models.load_model('best_model_acc')

for i in range (0,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

60669/60669 [==============================] - 782s 13ms/step
Matriz de Confusão Teste:  0
[[ 489859   10141]
 [  32367 1409014]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9380    0.9797    0.9584    500000
           1     0.9929    0.9775    0.9851   1441381

    accuracy                         0.9781   1941381
   macro avg     0.9654    0.9786    0.9718   1941381
weighted avg     0.9787    0.9781    0.9783   1941381

____________________
57413/57413 [==============================] - 711s 12ms/step
Matriz de Confusão Teste:  1
[[ 489859   10141]
 [  33004 1304194]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9369    0.9797    0.9578    500000
           1     0.9923    0.9753    0.9837   1337198

    accuracy                         0.9765   1837198
   macro avg     0.9646    0.9775    0.9708   1837198
weighted avg     0.9772    0.9765    0.9767   1837198

___________

## Teste Primeiro Treinamento Incremental

In [20]:
model_bin = tf.keras.models.load_model('best_model_acc')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste0.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase0',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7766/7766 [==============================] - ETA: 0s - loss: 0.0289 - accuracy: 0.9907INFO:tensorflow:Assets written to: best_model_acc_fase0/assets


INFO:tensorflow:Assets written to: best_model_acc_fase0/assets


7766/7766 [==============================] - 318s 41ms/step - loss: 0.0289 - accuracy: 0.9907 - val_loss: 0.0680 - val_accuracy: 0.9722
Epoch 2/10
7765/7766 [============================>.] - ETA: 0s - loss: 0.0214 - accuracy: 0.9934INFO:tensorflow:Assets written to: best_model_acc_fase0/assets


INFO:tensorflow:Assets written to: best_model_acc_fase0/assets


7766/7766 [==============================] - 314s 40ms/step - loss: 0.0214 - accuracy: 0.9934 - val_loss: 0.0521 - val_accuracy: 0.9803
Epoch 3/10
7765/7766 [============================>.] - ETA: 0s - loss: 0.0172 - accuracy: 0.9948INFO:tensorflow:Assets written to: best_model_acc_fase0/assets


...

7766/7766 [==============================] - 316s 41ms/step - loss: 0.0091 - accuracy: 0.9975 - val_loss: 0.0840 - val_accuracy: 0.9772
Epoch 9/10
7765/7766 [============================>.] - ETA: 0s - loss: 0.0082 - accuracy: 0.9978INFO:tensorflow:Assets written to: best_model_acc_fase0/assets


INFO:tensorflow:Assets written to: best_model_acc_fase0/assets


7766/7766 [==============================] - 316s 41ms/step - loss: 0.0082 - accuracy: 0.9978 - val_loss: 0.1029 - val_accuracy: 0.9738
Epoch 10/10
7765/7766 [============================>.] - ETA: 0s - loss: 0.0078 - accuracy: 0.9979INFO:tensorflow:Assets written to: best_model_acc_fase0/assets


INFO:tensorflow:Assets written to: best_model_acc_fase0/assets


7766/7766 [==============================] - 316s 41ms/step - loss: 0.0078 - accuracy: 0.9979 - val_loss: 0.0848 - val_accuracy: 0.9780


In [21]:
model_bin = tf.keras.models.load_model('best_model_acc_fase0')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (1,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

57413/57413 [==============================] - 714s 12ms/step
Matriz de Confusão Teste:  1
[[ 484833   15167]
 [  17296 1319902]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9656    0.9697    0.9676    500000
           1     0.9886    0.9871    0.9879   1337198

    accuracy                         0.9823   1837198
   macro avg     0.9771    0.9784    0.9777   1837198
weighted avg     0.9824    0.9823    0.9823   1837198

____________________
60792/60792 [==============================] - 728s 12ms/step
Matriz de Confusão Teste:  2
[[ 484833   15167]
 [  21108 1424206]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9583    0.9697    0.9639    500000
           1     0.9895    0.9854    0.9874   1445314

    accuracy                         0.9814   1945314
   macro avg     0.9739    0.9775    0.9757   1945314
weighted avg     0.9814    0.9814    0.9814   1945314

___________

## Teste Segundo Treinamento Incremental

In [22]:
model_bin = tf.keras.models.load_model('best_model_acc_fase0')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste1.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase1',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7347/7349 [============================>.] - ETA: 0s - loss: 0.0234 - accuracy: 0.9928

2024-01-22 22:32:05.226878: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.19GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


7349/7349 [==============================] - ETA: 0s - loss: 0.0234 - accuracy: 0.9928INFO:tensorflow:Assets written to: best_model_acc_fase1/assets


INFO:tensorflow:Assets written to: best_model_acc_fase1/assets


7349/7349 [==============================] - 301s 41ms/step - loss: 0.0234 - accuracy: 0.9928 - val_loss: 0.0409 - val_accuracy: 0.9840
Epoch 2/10
7348/7349 [============================>.] - ETA: 0s - loss: 0.0163 - accuracy: 0.9953INFO:tensorflow:Assets written to: best_model_acc_fase1/assets


...

7349/7349 [==============================] - 298s 41ms/step - loss: 0.0069 - accuracy: 0.9982 - val_loss: 0.0754 - val_accuracy: 0.9811
Epoch 9/10
7349/7349 [==============================] - ETA: 0s - loss: 0.0065 - accuracy: 0.9983INFO:tensorflow:Assets written to: best_model_acc_fase1/assets


INFO:tensorflow:Assets written to: best_model_acc_fase1/assets


7349/7349 [==============================] - 299s 41ms/step - loss: 0.0065 - accuracy: 0.9983 - val_loss: 0.0875 - val_accuracy: 0.9794
Epoch 10/10
7349/7349 [==============================] - ETA: 0s - loss: 0.0063 - accuracy: 0.9984INFO:tensorflow:Assets written to: best_model_acc_fase1/assets


INFO:tensorflow:Assets written to: best_model_acc_fase1/assets


7349/7349 [==============================] - 298s 41ms/step - loss: 0.0063 - accuracy: 0.9984 - val_loss: 0.0665 - val_accuracy: 0.9845


In [23]:
model_bin = tf.keras.models.load_model('best_model_acc_fase1')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (2,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

60792/60792 [==============================] - 754s 12ms/step
Matriz de Confusão Teste:  2
[[ 480664   19336]
 [  13858 1431456]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9720    0.9613    0.9666    500000
           1     0.9867    0.9904    0.9885   1445314

    accuracy                         0.9829   1945314
   macro avg     0.9793    0.9759    0.9776   1945314
weighted avg     0.9829    0.9829    0.9829   1945314

____________________
56590/56590 [==============================] - 679s 12ms/step
Matriz de Confusão Teste:  3
[[ 480664   19336]
 [  18674 1292193]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9626    0.9613    0.9620    500000
           1     0.9853    0.9858    0.9855   1310867

    accuracy                         0.9790   1810867
   macro avg     0.9739    0.9735    0.9737   1810867
weighted avg     0.9790    0.9790    0.9790   1810867

___________

## Teste Terceiro Treino Incremental

In [24]:
model_bin = tf.keras.models.load_model('best_model_acc_fase1')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste2.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase2',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7782/7782 [==============================] - ETA: 0s - loss: 0.0197 - accuracy: 0.9943INFO:tensorflow:Assets written to: best_model_acc_fase2/assets


INFO:tensorflow:Assets written to: best_model_acc_fase2/assets


7782/7782 [==============================] - 318s 41ms/step - loss: 0.0197 - accuracy: 0.9943 - val_loss: 0.0324 - val_accuracy: 0.9883
Epoch 2/10
7782/7782 [==============================] - ETA: 0s - loss: 0.0131 - accuracy: 0.9965INFO:tensorflow:Assets written to: best_model_acc_fase2/assets


INFO:tensorflow:Assets written to: best_model_acc_fase2/assets


7782/7782 [==============================] - 314s 40ms/step - loss: 0.0131 - accuracy: 0.9965 - val_loss: 0.0474 - val_accuracy: 0.9851
Epoch 3/10
7781/7782 [============================>.] - ETA: 0s - loss: 0.0101 - accuracy: 0.9974INFO:tensorflow:Assets written to: best_model_acc_fase2/assets


...

7782/7782 [==============================] - 316s 41ms/step - loss: 0.0058 - accuracy: 0.9986 - val_loss: 0.0741 - val_accuracy: 0.9826
Epoch 9/10
7781/7782 [============================>.] - ETA: 0s - loss: 0.0055 - accuracy: 0.9987INFO:tensorflow:Assets written to: best_model_acc_fase2/assets


INFO:tensorflow:Assets written to: best_model_acc_fase2/assets


7782/7782 [==============================] - 316s 41ms/step - loss: 0.0055 - accuracy: 0.9987 - val_loss: 0.0862 - val_accuracy: 0.9800
Epoch 10/10
7781/7782 [============================>.] - ETA: 0s - loss: 0.0052 - accuracy: 0.9988INFO:tensorflow:Assets written to: best_model_acc_fase2/assets


INFO:tensorflow:Assets written to: best_model_acc_fase2/assets


7782/7782 [==============================] - 315s 41ms/step - loss: 0.0052 - accuracy: 0.9988 - val_loss: 0.0652 - val_accuracy: 0.9856


In [25]:
model_bin = tf.keras.models.load_model('best_model_acc_fase2')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (3,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

56590/56590 [==============================] - 707s 12ms/step
Matriz de Confusão Teste:  3
[[ 481224   18776]
 [  15359 1295508]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9691    0.9624    0.9657    500000
           1     0.9857    0.9883    0.9870   1310867

    accuracy                         0.9811   1810867
   macro avg     0.9774    0.9754    0.9764   1810867
weighted avg     0.9811    0.9811    0.9811   1810867

____________________
61423/61423 [==============================] - 742s 12ms/step
Matriz de Confusão Teste:  4
[[ 481224   18776]
 [  19670 1445837]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9607    0.9624    0.9616    500000
           1     0.9872    0.9866    0.9869   1465507

    accuracy                         0.9804   1965507
   macro avg     0.9740    0.9745    0.9742   1965507
weighted avg     0.9805    0.9804    0.9804   1965507

___________

## Teste Quarto Treino Incremental

In [26]:
model_bin = tf.keras.models.load_model('best_model_acc_fase2')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste3.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase3',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7244/7244 [==============================] - ETA: 0s - loss: 0.0202 - accuracy: 0.9942INFO:tensorflow:Assets written to: best_model_acc_fase3/assets


INFO:tensorflow:Assets written to: best_model_acc_fase3/assets


7244/7244 [==============================] - 296s 41ms/step - loss: 0.0202 - accuracy: 0.9942 - val_loss: 0.0359 - val_accuracy: 0.9870
Epoch 2/10
7243/7244 [============================>.] - ETA: 0s - loss: 0.0127 - accuracy: 0.9967INFO:tensorflow:Assets written to: best_model_acc_fase3/assets


INFO:tensorflow:Assets written to: best_model_acc_fase3/assets


7244/7244 [==============================] - 292s 40ms/step - loss: 0.0127 - accuracy: 0.9967 - val_loss: 0.0307 - val_accuracy: 0.9903
Epoch 3/10
7244/7244 [==============================] - ETA: 0s - loss: 0.0096 - accuracy: 0.9976INFO:tensorflow:Assets written to: best_model_acc_fase3/assets


...

7244/7244 [==============================] - 293s 41ms/step - loss: 0.0054 - accuracy: 0.9987 - val_loss: 0.0737 - val_accuracy: 0.9832
Epoch 9/10
7243/7244 [============================>.] - ETA: 0s - loss: 0.0051 - accuracy: 0.9988INFO:tensorflow:Assets written to: best_model_acc_fase3/assets


INFO:tensorflow:Assets written to: best_model_acc_fase3/assets


7244/7244 [==============================] - 293s 40ms/step - loss: 0.0051 - accuracy: 0.9988 - val_loss: 0.0596 - val_accuracy: 0.9862
Epoch 10/10
7243/7244 [============================>.] - ETA: 0s - loss: 0.0048 - accuracy: 0.9989INFO:tensorflow:Assets written to: best_model_acc_fase3/assets


INFO:tensorflow:Assets written to: best_model_acc_fase3/assets


7244/7244 [==============================] - 293s 40ms/step - loss: 0.0048 - accuracy: 0.9989 - val_loss: 0.0825 - val_accuracy: 0.9831


In [27]:
model_bin = tf.keras.models.load_model('best_model_acc_fase3')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (4,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

61423/61423 [==============================] - 754s 12ms/step
Matriz de Confusão Teste:  4
[[ 482756   17244]
 [  15700 1449807]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9685    0.9655    0.9670    500000
           1     0.9882    0.9893    0.9888   1465507

    accuracy                         0.9832   1965507
   macro avg     0.9784    0.9774    0.9779   1965507
weighted avg     0.9832    0.9832    0.9832   1965507

____________________
56719/56719 [==============================] - 679s 12ms/step
Matriz de Confusão Teste:  5
[[ 482756   17244]
 [  16519 1298469]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9669    0.9655    0.9662    500000
           1     0.9869    0.9874    0.9872   1314988

    accuracy                         0.9814   1814988
   macro avg     0.9769    0.9765    0.9767   1814988
weighted avg     0.9814    0.9814    0.9814   1814988

___________

## Teste Quinto Treino Incremental

In [28]:
model_bin = tf.keras.models.load_model('best_model_acc_fase3')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste4.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase4',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7863/7863 [==============================] - ETA: 0s - loss: 0.0173 - accuracy: 0.9953INFO:tensorflow:Assets written to: best_model_acc_fase4/assets


INFO:tensorflow:Assets written to: best_model_acc_fase4/assets


7863/7863 [==============================] - 320s 40ms/step - loss: 0.0173 - accuracy: 0.9953 - val_loss: 0.0361 - val_accuracy: 0.9872
Epoch 2/10
7863/7863 [==============================] - ETA: 0s - loss: 0.0109 - accuracy: 0.9973INFO:tensorflow:Assets written to: best_model_acc_fase4/assets


INFO:tensorflow:Assets written to: best_model_acc_fase4/assets


7863/7863 [==============================] - 316s 40ms/step - loss: 0.0109 - accuracy: 0.9973 - val_loss: 0.0390 - val_accuracy: 0.9880
Epoch 3/10
7863/7863 [==============================] - ETA: 0s - loss: 0.0083 - accuracy: 0.9980INFO:tensorflow:Assets written to: best_model_acc_fase4/assets


...

7863/7863 [==============================] - 318s 40ms/step - loss: 0.0047 - accuracy: 0.9990 - val_loss: 0.0724 - val_accuracy: 0.9847
Epoch 9/10
7863/7863 [==============================] - ETA: 0s - loss: 0.0044 - accuracy: 0.9990INFO:tensorflow:Assets written to: best_model_acc_fase4/assets


INFO:tensorflow:Assets written to: best_model_acc_fase4/assets


7863/7863 [==============================] - 318s 40ms/step - loss: 0.0044 - accuracy: 0.9990 - val_loss: 0.0652 - val_accuracy: 0.9861
Epoch 10/10
7861/7863 [============================>.] - ETA: 0s - loss: 0.0042 - accuracy: 0.9991INFO:tensorflow:Assets written to: best_model_acc_fase4/assets


INFO:tensorflow:Assets written to: best_model_acc_fase4/assets


7863/7863 [==============================] - 318s 40ms/step - loss: 0.0042 - accuracy: 0.9991 - val_loss: 0.0751 - val_accuracy: 0.9852


In [29]:
model_bin = tf.keras.models.load_model('best_model_acc_fase4')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (5,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

56719/56719 [==============================] - 695s 12ms/step
Matriz de Confusão Teste:  5
[[ 482719   17281]
 [  14749 1300239]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9704    0.9654    0.9679    500000
           1     0.9869    0.9888    0.9878   1314988

    accuracy                         0.9824   1814988
   macro avg     0.9786    0.9771    0.9779   1814988
weighted avg     0.9823    0.9824    0.9823   1814988

____________________
60739/60739 [==============================] - 718s 12ms/step
Matriz de Confusão Teste:  6
[[ 482719   17281]
 [  16755 1426878]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9665    0.9654    0.9659    500000
           1     0.9880    0.9884    0.9882   1443633

    accuracy                         0.9825   1943633
   macro avg     0.9772    0.9769    0.9771   1943633
weighted avg     0.9825    0.9825    0.9825   1943633

___________

## Teste Sexto Treino Incremental

In [30]:
model_bin = tf.keras.models.load_model('best_model_acc_fase4')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste5.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase5',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7260/7260 [==============================] - ETA: 0s - loss: 0.0175 - accuracy: 0.9952INFO:tensorflow:Assets written to: best_model_acc_fase5/assets


INFO:tensorflow:Assets written to: best_model_acc_fase5/assets


7260/7260 [==============================] - 297s 41ms/step - loss: 0.0175 - accuracy: 0.9952 - val_loss: 0.0266 - val_accuracy: 0.9911
Epoch 2/10
7259/7260 [============================>.] - ETA: 0s - loss: 0.0107 - accuracy: 0.9974INFO:tensorflow:Assets written to: best_model_acc_fase5/assets


INFO:tensorflow:Assets written to: best_model_acc_fase5/assets


7260/7260 [==============================] - 292s 40ms/step - loss: 0.0107 - accuracy: 0.9974 - val_loss: 0.0302 - val_accuracy: 0.9911
Epoch 3/10
7259/7260 [============================>.] - ETA: 0s - loss: 0.0081 - accuracy: 0.9981INFO:tensorflow:Assets written to: best_model_acc_fase5/assets


...

7260/7260 [==============================] - 297s 41ms/step - loss: 0.0045 - accuracy: 0.9990 - val_loss: 0.0470 - val_accuracy: 0.9894
Epoch 9/10
7259/7260 [============================>.] - ETA: 0s - loss: 0.0043 - accuracy: 0.9991INFO:tensorflow:Assets written to: best_model_acc_fase5/assets


INFO:tensorflow:Assets written to: best_model_acc_fase5/assets


7260/7260 [==============================] - 297s 41ms/step - loss: 0.0043 - accuracy: 0.9991 - val_loss: 0.0782 - val_accuracy: 0.9840
Epoch 10/10
7259/7260 [============================>.] - ETA: 0s - loss: 0.0042 - accuracy: 0.9991INFO:tensorflow:Assets written to: best_model_acc_fase5/assets


INFO:tensorflow:Assets written to: best_model_acc_fase5/assets


7260/7260 [==============================] - 296s 41ms/step - loss: 0.0042 - accuracy: 0.9991 - val_loss: 0.0461 - val_accuracy: 0.9904


In [31]:
model_bin = tf.keras.models.load_model('best_model_acc_fase5')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (6,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

60739/60739 [==============================] - 761s 13ms/step
Matriz de Confusão Teste:  6
[[ 476087   23913]
 [   9151 1434482]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9811    0.9522    0.9664    500000
           1     0.9836    0.9937    0.9886   1443633

    accuracy                         0.9830   1943633
   macro avg     0.9824    0.9729    0.9775   1943633
weighted avg     0.9830    0.9830    0.9829   1943633

____________________
56582/56582 [==============================] - 682s 12ms/step
Matriz de Confusão Teste:  7
[[ 476087   23913]
 [  10322 1300300]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9788    0.9522    0.9653    500000
           1     0.9819    0.9921    0.9870   1310622

    accuracy                         0.9811   1810622
   macro avg     0.9804    0.9721    0.9762   1810622
weighted avg     0.9811    0.9811    0.9810   1810622

___________

## Sétimo Treino Incremental

In [32]:
model_bin = tf.keras.models.load_model('best_model_acc_fase5')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste6.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase6',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7775/7775 [==============================] - ETA: 0s - loss: 0.0157 - accuracy: 0.9959INFO:tensorflow:Assets written to: best_model_acc_fase6/assets


INFO:tensorflow:Assets written to: best_model_acc_fase6/assets


7775/7775 [==============================] - 319s 41ms/step - loss: 0.0157 - accuracy: 0.9959 - val_loss: 0.0334 - val_accuracy: 0.9885
Epoch 2/10
7775/7775 [==============================] - ETA: 0s - loss: 0.0098 - accuracy: 0.9976INFO:tensorflow:Assets written to: best_model_acc_fase6/assets


INFO:tensorflow:Assets written to: best_model_acc_fase6/assets


7775/7775 [==============================] - 315s 41ms/step - loss: 0.0098 - accuracy: 0.9976 - val_loss: 0.0353 - val_accuracy: 0.9890
Epoch 3/10
7774/7775 [============================>.] - ETA: 0s - loss: 0.0073 - accuracy: 0.9983INFO:tensorflow:Assets written to: best_model_acc_fase6/assets


...

7775/7775 [==============================] - 319s 41ms/step - loss: 0.0044 - accuracy: 0.9990 - val_loss: 0.0624 - val_accuracy: 0.9862
Epoch 8/10
7775/7775 [==============================] - ETA: 0s - loss: 0.0040 - accuracy: 0.9992INFO:tensorflow:Assets written to: best_model_acc_fase6/assets


INFO:tensorflow:Assets written to: best_model_acc_fase6/assets


7775/7775 [==============================] - 319s 41ms/step - loss: 0.0040 - accuracy: 0.9992 - val_loss: 0.0708 - val_accuracy: 0.9860
Epoch 9/10
7775/7775 [==============================] - 314s 40ms/step - loss: 0.0040 - accuracy: 0.9991 - val_loss: 0.0646 - val_accuracy: 0.9875
Epoch 10/10
7775/7775 [==============================] - ETA: 0s - loss: 0.0039 - accuracy: 0.9992INFO:tensorflow:Assets written to: best_model_acc_fase6/assets


INFO:tensorflow:Assets written to: best_model_acc_fase6/assets


7775/7775 [==============================] - 314s 40ms/step - loss: 0.0039 - accuracy: 0.9992 - val_loss: 0.0727 - val_accuracy: 0.9855


In [8]:
model_bin = tf.keras.models.load_model('best_model_acc_fase6')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (7,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

2024-01-23 11:54:36.858123: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-01-23 11:54:36.858190: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: r2d2
2024-01-23 11:54:36.858194: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: r2d2
2024-01-23 11:54:36.858410: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.154.5
2024-01-23 11:54:36.858435: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.146.2
2024-01-23 11:54:36.858440: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 535.146.2 does not match DSO version 535.154.5 -- cannot find working devices in this configuration


56582/56582 [==============================] - 629s 11ms/step
Matriz de Confusão Teste:  7
[[ 480942   19058]
 [  11698 1298924]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9763    0.9619    0.9690    500000
           1     0.9855    0.9911    0.9883   1310622

    accuracy                         0.9830   1810622
   macro avg     0.9809    0.9765    0.9787   1810622
weighted avg     0.9830    0.9830    0.9830   1810622

____________________
61418/61418 [==============================] - 678s 11ms/step
Matriz de Confusão Teste:  8
[[ 480942   19058]
 [  13561 1451792]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9726    0.9619    0.9672    500000
           1     0.9870    0.9907    0.9889   1465353

    accuracy                         0.9834   1965353
   macro avg     0.9798    0.9763    0.9780   1965353
weighted avg     0.9834    0.9834    0.9834   1965353

___________

## Oitavo Treino Incremental

In [8]:
model_bin = tf.keras.models.load_model('best_model_acc_fase6')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste7.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase7',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

2024-01-23 13:54:19.028187: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-23 13:54:19.028657: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-23 13:54:19.028948: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/10


2024-01-23 13:54:29.441243: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2024-01-23 13:54:30.427951: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-01-23 13:54:30.735429: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f3eabaa9f50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-23 13:54:30.735452: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-01-23 13:54:30.766254: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-23 13:54:30.969650: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

7243/7243 [==============================] - ETA: 0s - loss: 0.0152 - accuracy: 0.9960

2024-01-23 13:59:06.243206: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.73GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-23 13:59:06.248761: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.73GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


...

7243/7243 [==============================] - 293s 40ms/step - loss: 0.0038 - accuracy: 0.9992 - val_loss: 0.0690 - val_accuracy: 0.9862
Epoch 9/10
7243/7243 [==============================] - ETA: 0s - loss: 0.0038 - accuracy: 0.9992INFO:tensorflow:Assets written to: best_model_acc_fase7/assets


INFO:tensorflow:Assets written to: best_model_acc_fase7/assets


7243/7243 [==============================] - 292s 40ms/step - loss: 0.0038 - accuracy: 0.9992 - val_loss: 0.0667 - val_accuracy: 0.9866
Epoch 10/10
7243/7243 [==============================] - ETA: 0s - loss: 0.0036 - accuracy: 0.9993INFO:tensorflow:Assets written to: best_model_acc_fase7/assets


INFO:tensorflow:Assets written to: best_model_acc_fase7/assets


7243/7243 [==============================] - 292s 40ms/step - loss: 0.0036 - accuracy: 0.9993 - val_loss: 0.0702 - val_accuracy: 0.9862


In [9]:
model_bin = tf.keras.models.load_model('best_model_acc_fase7')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (8,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

61418/61418 [==============================] - 761s 12ms/step
Matriz de Confusão Teste:  8
[[ 481409   18591]
 [  13178 1452175]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9734    0.9628    0.9681    500000
           1     0.9874    0.9910    0.9892   1465353

    accuracy                         0.9838   1965353
   macro avg     0.9804    0.9769    0.9786   1965353
weighted avg     0.9838    0.9838    0.9838   1965353

____________________
56730/56730 [==============================] - 719s 13ms/step
Matriz de Confusão Teste:  9
[[ 481409   18591]
 [  14466 1300879]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9708    0.9628    0.9668    500000
           1     0.9859    0.9890    0.9875   1315345

    accuracy                         0.9818   1815345
   macro avg     0.9784    0.9759    0.9771   1815345
weighted avg     0.9818    0.9818    0.9818   1815345

___________

## Nono Treino Incremental

In [10]:
model_bin = tf.keras.models.load_model('best_model_acc_fase7')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste8.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase8',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7862/7862 [==============================] - ETA: 0s - loss: 0.0150 - accuracy: 0.9961INFO:tensorflow:Assets written to: best_model_acc_fase8/assets


INFO:tensorflow:Assets written to: best_model_acc_fase8/assets


7862/7862 [==============================] - 321s 40ms/step - loss: 0.0150 - accuracy: 0.9961 - val_loss: 0.0252 - val_accuracy: 0.9917
Epoch 2/10
7862/7862 [==============================] - ETA: 0s - loss: 0.0092 - accuracy: 0.9979INFO:tensorflow:Assets written to: best_model_acc_fase8/assets


INFO:tensorflow:Assets written to: best_model_acc_fase8/assets


7862/7862 [==============================] - 317s 40ms/step - loss: 0.0092 - accuracy: 0.9979 - val_loss: 0.0303 - val_accuracy: 0.9907
Epoch 3/10
7861/7862 [============================>.] - ETA: 0s - loss: 0.0070 - accuracy: 0.9984INFO:tensorflow:Assets written to: best_model_acc_fase8/assets


...

7862/7862 [==============================] - 319s 41ms/step - loss: 0.0038 - accuracy: 0.9992 - val_loss: 0.0460 - val_accuracy: 0.9900
Epoch 9/10
7861/7862 [============================>.] - ETA: 0s - loss: 0.0036 - accuracy: 0.9993INFO:tensorflow:Assets written to: best_model_acc_fase8/assets


INFO:tensorflow:Assets written to: best_model_acc_fase8/assets


7862/7862 [==============================] - 320s 41ms/step - loss: 0.0036 - accuracy: 0.9993 - val_loss: 0.0601 - val_accuracy: 0.9881
Epoch 10/10
7861/7862 [============================>.] - ETA: 0s - loss: 0.0036 - accuracy: 0.9993INFO:tensorflow:Assets written to: best_model_acc_fase8/assets


INFO:tensorflow:Assets written to: best_model_acc_fase8/assets


7862/7862 [==============================] - 319s 41ms/step - loss: 0.0036 - accuracy: 0.9993 - val_loss: 0.0700 - val_accuracy: 0.9859


In [11]:
model_bin = tf.keras.models.load_model('best_model_acc_fase8')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (9,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

56730/56730 [==============================] - 717s 13ms/step
Matriz de Confusão Teste:  9
[[ 479459   20541]
 [  12453 1302892]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9747    0.9589    0.9667    500000
           1     0.9845    0.9905    0.9875   1315345

    accuracy                         0.9818   1815345
   macro avg     0.9796    0.9747    0.9771   1815345
weighted avg     0.9818    0.9818    0.9818   1815345

____________________
61541/61541 [==============================] - 754s 12ms/step
Matriz de Confusão Teste:  10
[[ 479459   20541]
 [  15881 1453404]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9679    0.9589    0.9634    500000
           1     0.9861    0.9892    0.9876   1469285

    accuracy                         0.9815   1969285
   macro avg     0.9770    0.9741    0.9755   1969285
weighted avg     0.9815    0.9815    0.9815   1969285

__________

## Décimo treino Incremental

In [12]:
model_bin = tf.keras.models.load_model('best_model_acc_fase8')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste9.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase9',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7262/7262 [==============================] - ETA: 0s - loss: 0.0146 - accuracy: 0.9962INFO:tensorflow:Assets written to: best_model_acc_fase9/assets


INFO:tensorflow:Assets written to: best_model_acc_fase9/assets


7262/7262 [==============================] - 296s 40ms/step - loss: 0.0146 - accuracy: 0.9962 - val_loss: 0.0303 - val_accuracy: 0.9897
Epoch 2/10
7262/7262 [==============================] - ETA: 0s - loss: 0.0090 - accuracy: 0.9979INFO:tensorflow:Assets written to: best_model_acc_fase9/assets


INFO:tensorflow:Assets written to: best_model_acc_fase9/assets


7262/7262 [==============================] - 292s 40ms/step - loss: 0.0090 - accuracy: 0.9979 - val_loss: 0.0319 - val_accuracy: 0.9906
Epoch 3/10
7261/7262 [============================>.] - ETA: 0s - loss: 0.0067 - accuracy: 0.9985INFO:tensorflow:Assets written to: best_model_acc_fase9/assets


...

7262/7262 [==============================] - 295s 41ms/step - loss: 0.0038 - accuracy: 0.9992 - val_loss: 0.0676 - val_accuracy: 0.9863
Epoch 9/10
7261/7262 [============================>.] - ETA: 0s - loss: 0.0036 - accuracy: 0.9993INFO:tensorflow:Assets written to: best_model_acc_fase9/assets


INFO:tensorflow:Assets written to: best_model_acc_fase9/assets


7262/7262 [==============================] - 295s 41ms/step - loss: 0.0036 - accuracy: 0.9993 - val_loss: 0.0573 - val_accuracy: 0.9874
Epoch 10/10
7261/7262 [============================>.] - ETA: 0s - loss: 0.0035 - accuracy: 0.9993INFO:tensorflow:Assets written to: best_model_acc_fase9/assets


INFO:tensorflow:Assets written to: best_model_acc_fase9/assets


7262/7262 [==============================] - 293s 40ms/step - loss: 0.0035 - accuracy: 0.9993 - val_loss: 0.0553 - val_accuracy: 0.9890


In [13]:
model_bin = tf.keras.models.load_model('best_model_acc_fase9')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (10,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

61541/61541 [==============================] - 765s 12ms/step
Matriz de Confusão Teste:  10
[[ 477177   22823]
 [  10135 1459150]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9792    0.9544    0.9666    500000
           1     0.9846    0.9931    0.9888   1469285

    accuracy                         0.9833   1969285
   macro avg     0.9819    0.9737    0.9777   1969285
weighted avg     0.9832    0.9833    0.9832   1969285

____________________
57414/57414 [==============================] - 696s 12ms/step
Matriz de Confusão Teste:  11
[[ 477177   22823]
 [  11582 1325655]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9763    0.9544    0.9652    500000
           1     0.9831    0.9913    0.9872   1337237

    accuracy                         0.9813   1837237
   macro avg     0.9797    0.9728    0.9762   1837237
weighted avg     0.9812    0.9813    0.9812   1837237

_________

## Décimo primeiro treino Incremental

In [7]:
model_bin = tf.keras.models.load_model('best_model_acc_fase9')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste10.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc_fase10',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

2024-01-24 08:19:32.997026: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-24 08:19:33.150811: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-24 08:19:33.150981: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/10


2024-01-24 08:19:43.381116: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2024-01-24 08:19:44.159554: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-01-24 08:19:44.178737: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x563b2bcb9690 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-24 08:19:44.178773: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-01-24 08:19:44.214868: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-24 08:19:44.434186: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

7878/7878 [==============================] - ETA: 0s - loss: 0.0142 - accuracy: 0.9963

2024-01-24 08:24:43.739666: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.47GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-24 08:24:44.804122: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.73GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-24 08:24:44.809807: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.73GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


...

7878/7878 [==============================] - 318s 40ms/step - loss: 0.0036 - accuracy: 0.9993 - val_loss: 0.0462 - val_accuracy: 0.9900
Epoch 9/10
7877/7878 [============================>.] - ETA: 0s - loss: 0.0035 - accuracy: 0.9993INFO:tensorflow:Assets written to: best_model_acc_fase10/assets


INFO:tensorflow:Assets written to: best_model_acc_fase10/assets


7878/7878 [==============================] - 318s 40ms/step - loss: 0.0035 - accuracy: 0.9993 - val_loss: 0.0619 - val_accuracy: 0.9877
Epoch 10/10
7878/7878 [==============================] - ETA: 0s - loss: 0.0034 - accuracy: 0.9993INFO:tensorflow:Assets written to: best_model_acc_fase10/assets


INFO:tensorflow:Assets written to: best_model_acc_fase10/assets


7878/7878 [==============================] - 318s 40ms/step - loss: 0.0034 - accuracy: 0.9993 - val_loss: 0.0514 - val_accuracy: 0.9898


In [8]:
model_bin = tf.keras.models.load_model('best_model_acc_fase10')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (11,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

57414/57414 [==============================] - 714s 12ms/step
Matriz de Confusão Teste:  11
[[ 476869   23131]
 [   9225 1328012]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9810    0.9537    0.9672    500000
           1     0.9829    0.9931    0.9880   1337237

    accuracy                         0.9824   1837237
   macro avg     0.9820    0.9734    0.9776   1837237
weighted avg     0.9824    0.9824    0.9823   1837237

____________________
